<a href="https://colab.research.google.com/github/PsCushman/baseball-rule-changes/blob/PSC/batter_learning_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from google.colab import drive
drive.mount('/content/drive')

#  Import and read the charity_data.csv.
file_path = '/content/drive/My Drive/Colab_Notebooks/off_output_for_learning.csv'
batter_woba_df = pd.read_csv(file_path)
batter_woba_df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,IDfg,Name,Age,G,AB,PA,H,1B,2B,3B,...,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR,avg_wOBA,wOBA_2023,zscore_difference
0,1744,Miguel Cabrera,37.666667,126.000000,454.000000,502.666667,120.333333,94.000000,15.666667,0.000000,...,351.333333,0.152667,0.268000,0.246000,0.395667,0.307000,-0.766667,0.299333,0.293,0.055586
1,2136,David Peralta,32.666667,127.666667,436.000000,483.666667,113.666667,68.666667,29.666667,4.666667,...,341.333333,0.157000,0.266333,0.246000,0.374333,0.303333,1.266667,0.322333,0.311,-0.189938
2,2396,Carlos Santana,34.666667,149.000000,523.000000,617.000000,123.000000,77.666667,21.000000,0.333333,...,426.666667,0.166333,0.249000,0.254333,0.446333,0.351333,1.766667,0.327333,0.307,-0.449445
3,2434,Nelson Cruz,39.666667,128.000000,471.666667,537.333333,127.333333,78.333333,21.000000,0.333333,...,354.000000,0.127333,0.271667,0.265000,0.516000,0.366000,1.800000,0.351667,0.291,-1.622077
4,2967,Tommy Pham,32.666667,148.000000,532.000000,612.333333,131.666667,85.666667,26.666667,1.666667,...,396.000000,0.204333,0.289000,0.253667,0.425333,0.338333,1.733333,0.323667,0.343,0.602240


In [ ]:
# Drop the 'Name', IDfg, 'wOBA_2023' so it is not part of the classifacation and regression.
batter_woba_df.drop(['Name', 'IDfg', 'wOBA_2023'], axis=1, inplace=True)
batter_woba_df_encoded = batter_woba_df.copy()
batter_woba_df_encoded['zscore_difference'] = batter_woba_df_encoded['zscore_difference'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
batter_woba_df_encoded.dropna(axis=1, inplace=True)
batter_woba_df_encoded.head()


,Age,G,AB,PA,H,1B,2B,3B,HR,R,...,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR,avg_wOBA,zscore_difference
0,37.666667,126.000000,454.000000,502.666667,120.333333,94.000000,15.666667,0.000000,10.666667,38.0,...,0.438667,351.333333,0.152667,0.268000,0.246000,0.395667,0.307000,-0.766667,0.299333,1
1,32.666667,127.666667,436.000000,483.666667,113.666667,68.666667,29.666667,4.666667,10.666667,48.0,...,0.422000,341.333333,0.157000,0.266333,0.246000,0.374333,0.303333,1.266667,0.322333,0
2,34.666667,149.000000,523.000000,617.000000,123.000000,77.666667,21.000000,0.333333,24.000000,76.0,...,0.438667,426.666667,0.166333,0.249000,0.254333,0.446333,0.351333,1.766667,0.327333,0
3,39.666667,128.000000,471.666667,537.333333,127.333333,78.333333,21.000000,0.333333,27.666667,70.0,...,0.497000,354.000000,0.127333,0.271667,0.265000,0.516000,0.366000,1.800000,0.351667,0
4,32.666667,148.000000,532.000000,612.333333,131.666667,85.666667,26.666667,1.666667,17.666667,80.0,...,0.466000,396.000000,0.204333,0.289000,0.253667,0.425333,0.338333,1.733333,0.323667,1


In [ ]:
# Split our preprocessed data into our features and target arrays
X = batter_woba_df_encoded.drop('zscore_difference', axis=1)  # Features (remove the target column)
y = batter_woba_df_encoded['zscore_difference']              # Target

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the training and testing datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (243, 247)
X_test shape: (61, 247)
y_train shape: (243,)
y_test shape: (61,)


In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler on the training data
X_scaler = scaler.fit(X_train)

# Scale the training and test data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

print("Selected X_train shape:", X_train_scaled.shape)
print("Selected X_test shape:", X_test_scaled.shape)
print("Selected y_train shape:", y_train.shape)

Selected X_train shape: (243, 247)
Selected X_test shape: (61, 247)
Selected y_train shape: (243,)


In [ ]:
# Instantiate the random oversampler model
oversampler = RandomOverSampler(random_state=1)

# Fit the scaled training data to the random oversampler model
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_scaled, y_train)

In [ ]:
!pip install keras-tuner

In [ ]:
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch

In [ ]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=247))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [ ]:
tuner = kt.Hyperband(
    create_model,
    objective='accuracy',
    max_epochs=20,
    factor=3,
    hyperband_iterations=2,
    directory='my_hyperband_tuner_directory',
    project_name='my_hyperband_tuner_project'
)

# Assuming you've already set up your data (X_train_resampled, y_train_resampled, X_test_scaled, y_test)
tuner.search(X_train_resampled, y_train_resampled, epochs=20, validation_data=(X_test_scaled, y_test))

# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_model = create_model(best_hyper)

# Train the best model
best_model.fit(X_train_resampled, y_train_resampled, epochs=20, validation_data=(X_test_scaled, y_test))



Epoch 1/20
8/8 [==============================] - 2s 32ms/step - loss: 0.7133 - accuracy: 0.5000 - val_loss: 0.7041 - val_accuracy: 0.4918
Epoch 2/20
8/8 [==============================] - 0s 4ms/step - loss: 0.7010 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.4918
Epoch 3/20
8/8 [==============================] - 0s 4ms/step - loss: 0.6904 - accuracy: 0.5000 - val_loss: 0.6845 - val_accuracy: 0.4918
Epoch 4/20
8/8 [==============================] - 0s 4ms/step - loss: 0.6811 - accuracy: 0.5276 - val_loss: 0.6777 - val_accuracy: 0.5246
Epoch 5/20
8/8 [==============================] - 0s 4ms/step - loss: 0.6741 - accuracy: 0.5709 - val_loss: 0.6720 - val_accuracy: 0.5902
Epoch 6/20
8/8 [==============================] - 0s 5ms/step - loss: 0.6682 - accuracy: 0.6181 - val_loss: 0.6671 - val_accuracy: 0.6393
Epoch 7/20
8/8 [==============================] - 0s 5ms/step - loss: 0.6624 - accuracy: 0.6575 - val_loss: 0.6633 - val_accuracy: 0.7049
Epoch 8/20
8/8 [=================

In [ ]:
# # Run the kerastuner search for best hyperparameters
# tuner.search(X_train_resampled,y_train_resampled,epochs=20,validation_data=(X_test_scaled,y_test))

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = best_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 0.6371 - accuracy: 0.6885 - 16ms/epoch - 8ms/step
Loss: 0.6370784640312195, Accuracy: 0.688524603843689
